ETL (pair 11)

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import scipy.stats as stats
from scipy.stats import shapiro, poisson, chisquare, expon, kstest

In [43]:
df_clientes = pd.read_csv('../files/clientes.csv')
df_clientes.head(3)

,id,first_name,last_name,email,gender,City,Country,Address
0,1,Cheri,Dunsmore,cdunsmore0@instagram.com,Female,Palma De Mallorca,Spain,076 Rockefeller Crossing
1,2,Hunt,Bartomeu,hbartomeu1@nsw.gov.au,Male,Lugo,Spain,0046 Utah Junction
2,3,Michaeline,Paynton,mpaynton2@narod.ru,Female,NaN,Spain,0 Corry Crossing


In [13]:
df_ventas = pd.read_csv("../files/ventas.csv", index_col=0)
df_ventas.head(3)

,ID_Producto,Fecha_Venta,Cantidad,Total
ID_Cliente,,,,
723,A1,2023-11-22,2,17.98
498,C3,2023-11-21,1,5.49
121,D4,2023-11-20,3,32.97


In [30]:
# df = pd.read_csv("../files/productos.csv", engine="python")

df_productos = pd.read_csv("../files/productos.csv", delimiter=",")
df_productos.head(3)

,ID,Nombre_Producto,Categoría,Precio,Origen,Descripción
0,A1,Pizza Margherita,Platos Preparados,8.99,Italia,Clásica pizza italiana con tomate/mozzarella f...
1,B2,Risotto de Champiñones,Platos Preparados,6.75,Italia,Risotto cremoso/con champiñones frescos una de...
2,C3,Tiramisú,Postres,5.49,Italia,Postre clásico italiano con capas de bizcocho ...


In [70]:
def cambiar_a_unk(df,columnas):

    for c in columnas:
        try:
            df[c]= df[c].fillna('unknown')
        except:
            df[c]= df[c]

In [71]:
cambiar_a_unk(df_clientes,df_clientes.columns)

In [72]:
df_clientes.isnull().sum()

ID_Cliente    0
first_name    0
last_name     0
email         0
gender        0
City          0
Country       0
Address       0
dtype: int64

In [ ]:
df_productos.info() #no hay nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               33 non-null     object 
 1   Nombre_Producto  33 non-null     object 
 2   Categoría        33 non-null     object 
 3   Precio           33 non-null     float64
 4   Origen           33 non-null     object 
 5   Descripción      33 non-null     object 
dtypes: float64(1), object(5)
memory usage: 1.7+ KB


In [25]:
df_ventas.info() #no hay nulos

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 723 to 987
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID_Producto  100 non-null    object 
 1   Fecha_Venta  100 non-null    object 
 2   Cantidad     100 non-null    int64  
 3   Total        100 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 3.9+ KB


In [33]:
def buscar_duplicados(df):
    print(f'Este DataFrame tiene {df.duplicated().sum()} filas duplicadas')

In [34]:
buscar_duplicados(df_productos)

Este DataFrame tiene 0 filas duplicadas


In [31]:
df_productos['Descripción'] = df_productos['Descripción'].str.replace('/', ', ')

In [35]:
df_ventas.head(3)

,ID_Producto,Fecha_Venta,Cantidad,Total
ID_Cliente,,,,
723,A1,2023-11-22,2,17.98
498,C3,2023-11-21,1,5.49
121,D4,2023-11-20,3,32.97


In [45]:
df_clientes.rename(columns = {"id": "ID_Cliente"}, inplace = True )

In [46]:
df_clientes.head(3)

,ID_Cliente,first_name,last_name,email,gender,City,Country,Address
0,1,Cheri,Dunsmore,cdunsmore0@instagram.com,Female,Palma De Mallorca,Spain,076 Rockefeller Crossing
1,2,Hunt,Bartomeu,hbartomeu1@nsw.gov.au,Male,Lugo,Spain,0046 Utah Junction
2,3,Michaeline,Paynton,mpaynton2@narod.ru,Female,NaN,Spain,0 Corry Crossing


In [62]:
df_productos.head()

,ID,Nombre_Producto,Categoría,Precio,Origen,Descripción
0,A1,Pizza Margherita,Platos Preparados,8.99,Italia,"Clásica pizza italiana con tomate, mozzarella ..."
1,B2,Risotto de Champiñones,Platos Preparados,6.75,Italia,"Risotto cremoso, con champiñones frescos una d..."
2,C3,Tiramisú,Postres,5.49,Italia,Postre clásico italiano con capas de bizcocho ...
3,D4,Panettone,Repostería,10.99,Italia,Pan dulce navideño italiano con frutas confita...
4,E5,Orecchiette,Productos Secos,4.29,Italia,Pequeñas pastas con forma de orecchiette ideal...


In [73]:
df_join = (df_clientes.merge(df_ventas, how='inner', on='ID_Cliente')).merge(df_productos, how='left', left_on='ID_Producto', right_on='ID')

In [65]:
df_ventas.shape

(100, 4)

In [64]:
df_join.shape

(10, 18)

In [74]:
df_join.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID_Cliente       100 non-null    int64  
 1   first_name       100 non-null    object 
 2   last_name        100 non-null    object 
 3   email            100 non-null    object 
 4   gender           100 non-null    object 
 5   City             100 non-null    object 
 6   Country          100 non-null    object 
 7   Address          100 non-null    object 
 8   ID_Producto      100 non-null    object 
 9   Fecha_Venta      100 non-null    object 
 10  Cantidad         100 non-null    int64  
 11  Total            100 non-null    float64
 12  ID               10 non-null     object 
 13  Nombre_Producto  10 non-null     object 
 14  Categoría        10 non-null     object 
 15  Precio           10 non-null     float64
 16  Origen           10 non-null     object 
 17  Descripción      

In [76]:
df_join['Country'] = df_join['Country'].replace('unknown', 'Spain')

In [78]:
df_join['Country'].unique()

array(['Spain'], dtype=object)

In [77]:
df_join.sample(50)

,ID_Cliente,first_name,last_name,email,gender,City,Country,Address,ID_Producto,Fecha_Venta,Cantidad,Total,ID,Nombre_Producto,Categoría,Precio,Origen,Descripción
7,234,Anni,Handrock,ahandrock6h@vk.com,Female,Sevilla,Spain,10 Maple Road,BQ85,2023-10-25,2,9.98,NaN,NaN,NaN,NaN,NaN,NaN
32,321,Tynan,Oty,toty8w@nymag.com,Male,Albacete,Spain,02 Reindahl Terrace,AH126,2023-09-14,1,4.75,NaN,NaN,NaN,NaN,NaN,NaN
18,234,Anni,Handrock,ahandrock6h@vk.com,Female,Sevilla,Spain,10 Maple Road,CT114,2023-09-26,1,4.75,NaN,NaN,NaN,NaN,NaN,NaN
92,987,Cecil,Hemms,chemmsre@reverbnation.com,Male,unknown,Spain,1737 Commercial Lane,CM107,2023-10-03,2,12.50,NaN,NaN,NaN,NaN,NaN,NaN
90,987,Cecil,Hemms,chemmsre@reverbnation.com,Male,unknown,Spain,1737 Commercial Lane,CC97,2023-10-13,2,9.98,NaN,NaN,NaN,NaN,NaN,NaN
76,876,Mallory,Crallan,mcrallanob@issuu.com,Male,Murcia,Spain,87898 Hoepker Junction,CA95,2023-10-15,2,12.50,NaN,NaN,NaN,NaN,NaN,NaN
88,987,Cecil,Hemms,chemmsre@reverbnation.com,Male,unknown,Spain,1737 Commercial Lane,BR86,2023-10-24,1,5.25,NaN,NaN,NaN,NaN,NaN,NaN
10,234,Anni,Handrock,ahandrock6h@vk.com,Female,Sevilla,Spain,10 Maple Road,BZ94,2023-10-16,1,8.45,NaN,NaN,NaN,NaN,NaN,NaN
99,987,Cecil,Hemms,chemmsre@reverbnation.com,Male,unknown,Spain,1737 Commercial Lane,AV140,2023-08-31,1,5.25,NaN,NaN,NaN,NaN,NaN,NaN
52,543,Mano,Briat,mbriatf2@parallels.com,unknown,Sevilla,Spain,90 Bultman Street,AQ135,2023-09-05,3,26.97,NaN,NaN,NaN,NaN,NaN,NaN
